In [143]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import sys

from matplotlib import pyplot as plt
from skimage import color
from scipy.sparse import csr_matrix

import torch
from torch import optim
from torch.nn import functional as tf
from torchvision import models
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from utils import VOC_LABEL2COLOR
from utils import VOC_STATISTICS
from utils import numpy2torch
from utils import torch2numpy

In [144]:
# VOC Dataset class, derived from PyTorch Dataset
class VOC2007Dataset(Dataset):
    def __init__(self, root, train, num_examples):
        super().__init__()
        # Reading num_examples lines from the split file for the image names
        if train:
            split_file = open(root + "/ImageSets/Segmentation/train.txt", "r")
        else:
            split_file = open(root + "/ImageSets/Segmentation/val.txt", "r")
        splits = []
        # TODO: Read random line?
        for _ in range(0, num_examples):
            line = split_file.readline()
            splits.append(line[:6])
        split_file.close()
        
        # Reading pictures and gt for the entries in splits
        self.images = np.array([])
        self.labels = np.array([])
        for name in splits:
            im_file = open(root + "/JPEGImages/{}.jpg".format(name), "r")
            #print(type(im_file.read()))
            print(np.shape(im_file.read()))
            # self.images.append(im_file.read())
            gt_file = open(root + "/SegmentationClass/{}.png".format(name), "r")
            # self.labels = ...
        
    def __getitem__(self, index):
        example_dict = dict()
        assert (isinstance(example_dict, dict))
        assert ('im' in example_dict.keys())
        assert ('gt' in example_dict.keys())
        return example_dict

    def __len__(self):
        assert(self.images.size == self.labels.size)
        return self.images.size

In [145]:
# Create and return pytorch.utils DataLoader
def create_loader(dataset, batch_size, shuffle, num_workers):
    loader = DataLoader(dataset, batch_size, shuffle, num_workers=num_workers)
    assert (isinstance(loader, DataLoader))
    return loader

In [146]:
def voc_label2color(np_image, np_label):
    assert (isinstance(np_image, np.ndarray))
    assert (isinstance(np_label, np.ndarray))

    colored = []

    assert (np.equal(colored.shape, np_image.shape).all())
    assert (np_image.dtype == colored.dtype)
    return colored

In [147]:
def show_dataset_examples(loader, grid_height, grid_width, title):
    pass

In [148]:
def standardize_input(input_tensor):
    normalized = []

    assert (type(input_tensor) == type(normalized))
    assert (input_tensor.size() == normalized.size())
    return normalized

In [149]:
def run_forward_pass(normalized, model):
    prediction = []
    acts = []

    assert (isinstance(prediction, torch.Tensor))
    assert (isinstance(acts, torch.Tensor))
    return prediction, acts

In [150]:
def average_precision(prediction, gt):
    return None

In [151]:
def show_inference_examples(loader, model, grid_height, grid_width, title):
    pass

In [152]:
def find_unique_example(loader, unique_foreground_label):
    example = []

    assert (isinstance(example, dict))
    return example

In [153]:
def show_unique_example(example_dict, model):
    pass

In [154]:
def show_attack(example_dict, model, src_label, target_label, learning_rate, iterations):
    pass

In [155]:
def problem2():
    # Please set an environment variables 'VOC2007_HOME' pointing to your '../VOCdevkit/VOC2007' folder
    os.environ["VOC2007_HOME"] = "/home/yannik/Computer-Vison-2/asgn4/VOCdevkit/VOC2007"
    root = os.environ["VOC2007_HOME"]

    
    # create datasets for training and validation
    #train_dataset = VOC2007Dataset(root, train=True, num_examples=128)
    train_dataset = VOC2007Dataset(root, train=True, num_examples=2)
    #valid_dataset = VOC2007Dataset(root, train=False, num_examples=128)
    valid_dataset = VOC2007Dataset(root, train=False, num_examples=2)
    
    # create data loaders for training and validation
    # you can safely assume batch_size=1 in our tests..
    train_loader = create_loader(train_dataset, batch_size=1, shuffle=True, num_workers=1)
    valid_loader = create_loader(valid_dataset, batch_size=1, shuffle=False, num_workers=1)

    """
    # show some images for the training and validation set
    show_dataset_examples(train_loader, grid_height=2, grid_width=3, title='training examples')
    show_dataset_examples(valid_loader, grid_height=2, grid_width=3, title='validation examples')

    # Load Deeplab network
    model = models.segmentation.deeplabv3_resnet101(pretrained=True, num_classes=21)

    # Apply deeplab. Switch to training loader if you want more variety.
    show_inference_examples(valid_loader, model, grid_height=2, grid_width=3, title='inference examples')

    # attack1: convert cat to dog
    cat_example = find_unique_example(valid_loader, unique_foreground_label=8)
    show_unique_example(cat_example, model=model)
    show_attack(cat_example, model, src_label=8, target_label=12, learning_rate=1.0, iterations=10)
    """
    
    # feel free to try other examples..

In [156]:
if __name__ == '__main__':
    problem2()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte